# 01. Introduction

* **Objective**
  * Preparing the input data for the multimodal drug sensitivity prediction project
  * Becoming familiar with the data
  * Expected outcome: train_response data with ["celline id", "celline pc components",  "SMILES", "label"] columns

---
* **Datasets Overview**
  * `drug_response_cid.csv`: *drug response data file*
    * COSMIC.ID (cell line ID)
    * pubchem (drug id)
    * response (binary)
        --> 기준 : ifelse(drug.res$MAX_CONC > drug.res$IC50_PUBLISHED,1,0)
        --> 1: sensitive, 0: resistant
  * `exp_filtered`: *celline expression data file * 
    * log(TPM+1)
    * COSMIC.ID (cell line id)
    * continuous feature variables
  * `mut_filtered.csv`: *cell line mutation data file*
    * COSMIC.ID (cell line id)
    * Gene (gene name)
    * binary feature variables
  * `cell_info_filtered_662cell.csv`: *cell line information data file*
    * log(TPM+1)
    * COSMIC.ID (cell line id)
---


Specifically, we're going to implement:

| **Steps** | **Contents** |
| ----- | ----- |
| **1. Setting up the Environment** | We are going to use pandas, sklearn and pubchempy  |
| **2. Loading data** | To prepare input data for drug sensitivity prediction, we'll start with data that we preprocessed previously|
| **3. Data preproccessing** | We've got some data loaded, let's explore and process them|
| **4. Preparing chemical features** | Here we'll obtain the SMILES format for drugs using **pubchempy** library |
| **5. Merging chemical features** | Merging the chemical features with drug response data|
| **6. Saving the final result data** | Saving the results and check if it is correctly load back|
| **7. Optional: Preparing cell line features** | Let's make some dimension reduction of cellline genomics |
| **8. Optional: Mapping the features into final training data** | Merging the chemical features and cell line features |
| **9. Optional: Saving the final result data** | Since we want to use the data for next tutorial, let's save it and make sure it loads back correctly |
| **10. Homework** | Use reference function do same processes with drug bank data (drug_bank.csv). Which method is better ? How ?|

## Step 1: Setting up the Environment
* **Importing Libraries**
  * `pandas`: *data manipulation and analysis* ([link for pandas library documentation](https://pandas.pydata.org/docs/))
  * `numpy`: *library for large, multi-dimensional arrays and matrices* ([link for numpy library documentation](https://numpy.org/doc/stable/))
  * `sklearn`: *machine learning library to* ([link for scikit-learn library documentation](https://scikit-learn.org/stable/getting_started.html)) 
  * `pubchempy`: *chemical information from PubChem database* [link for pubchempy library documentation](https://pubchempy.readthedocs.io/en/latest/)
  * `pathlib`: *filesystem paths (no difference in OS, Windows, LINUX)* [link for pathlib library documentation](https://docs.python.org/3/library/pathlib.html)
  * `bokeh`: *data visualization* [link for pathlib library documentation](https://docs.bokeh.org/en/latest/)


In [3]:
import pandas as pd
import numpy as np
import pubchempy as pcp
from sklearn.decomposition import PCA
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem

pd.set_option('mode.chained_assignment',  None) # we dont want unnecessary warning

## Step 2: Loading Data
* **creating data paths**
  * `data_root`: *a directory where all hands-on files are in*
  * creating file paths


In [23]:
data_root = Path("./input_data")

drug_response_file = data_root.joinpath("drug_response_cid.csv")
exp_file = data_root.joinpath("exp_filtered.csv")
mut_file = data_root.joinpath("mut_filtered.csv")

print(drug_response_file)
print(exp_file)
print(mut_file)

input_data/drug_response_cid.csv
input_data/exp_filtered.csv
input_data/mut_filtered.csv


* **checking if path is correct and if file is exist**

In [27]:
print(drug_response_file.is_file())
print(exp_file.is_file())
print(mut_file.is_file())

True
True
True


* **load all data in to dataframes**

In [28]:
df_response = pd.read_csv(drug_response_file)
df_exp = pd.read_csv(exp_file)
df_mut = pd.read_csv(mut_file)

In [29]:
df_response

,COSMIC.ID,pubchem,response
0,909762,176870,0
1,910691,176870,0
2,910697,176870,1
3,1303901,176870,0
4,909248,176870,0
...,...,...,...
170197,749711,46907787,1
170198,906826,46907787,0
170199,906851,46907787,0
170200,925338,46907787,0


In [30]:
df_exp

,COSMIC.ID,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,H3C2,H3C3,AC098582.1,DUS4L.BCAP29,C8orf44.SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
0,684072,4.327687,0.070389,5.979339,2.906891,4.904484,0.263034,2.235727,0.422233,4.433627,...,1.416840,2.000000,0.042644,1.959770,0.432959,0.070389,0.042644,0.378512,0.070389,0.000000
1,687448,3.266037,0.000000,6.096979,2.521051,3.040892,0.000000,0.831877,6.576069,4.657068,...,0.333424,0.879706,0.839960,2.384050,0.000000,0.028569,0.163499,0.000000,0.028569,0.000000
2,687562,4.374344,0.000000,6.963821,2.292782,4.001802,0.495695,2.729009,6.396947,5.355792,...,2.056584,0.000000,0.333424,2.792855,0.584963,0.000000,0.238787,0.070389,0.000000,0.028569
3,687568,3.477677,0.000000,6.762615,2.107688,4.371559,0.028569,1.321928,5.440288,4.072963,...,1.910733,1.035624,0.367371,1.673556,0.097611,0.000000,0.042644,0.124328,0.000000,0.389567
4,687590,3.244887,0.000000,7.124535,1.933573,3.493135,0.014355,3.168321,6.090642,4.505891,...,1.695994,1.613532,0.594549,4.015248,0.286881,0.000000,0.028569,0.000000,0.000000,0.411426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,1330948,0.176323,0.000000,6.348197,2.280956,3.636915,1.831877,5.785551,5.497293,6.055065,...,1.937344,0.432959,0.454176,2.920293,0.189034,0.042644,0.014355,0.042644,0.042644,0.000000
658,1503364,4.585563,0.000000,7.524973,2.211012,2.786596,0.000000,0.310340,5.228819,4.791293,...,1.144046,0.782409,0.263034,1.220330,0.111031,0.000000,0.000000,0.000000,0.000000,0.000000
659,1659818,0.137504,0.000000,6.757423,3.176323,5.111449,0.097611,0.189034,3.688180,4.213347,...,0.650765,1.422233,0.604071,2.223423,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
660,1659819,0.000000,0.000000,7.059939,2.761285,4.144862,0.443607,0.000000,0.056584,4.491853,...,1.443607,1.906891,1.070389,1.448901,0.641546,0.000000,0.000000,0.000000,0.000000,0.000000


df_mut

In [ ]:
def df_na(df):
    # Select rows that contain at least one NA value
    rows_with_na = df[df.isnull().any(axis=1)]

    # Select columns that contain at least one NA value in those rows
    columns_with_na = rows_with_na.columns[rows_with_na.isnull().any()]

    # Show the selected rows and columns
    result = rows_with_na[columns_with_na]

    return result

## Step 3: Data Preprocessing

* **Getting drugs by pubchem (cid) from drug response data**
* **Createing dataframe that contains the drugs**

In [32]:
drug_list = df_response["pubchem"].unique()
drug_list[:10], len(drug_list)

(array([  176870,  5384616,  5329102, 10461815,   462382,    36314,
        11676786,   216239,  5494449,     5291]),
 307)

In [33]:
df_drugs = pd.DataFrame(data=drug_list, columns=["pubchem"])
df_drugs

,pubchem
0,176870
1,5384616
2,5329102
3,10461815
4,462382
...,...
302,16224745
303,118704762
304,91663328
305,117072552


## Step 3: Preparing chemical features

* **Getting SMILES for drugs using pubchempy libary throught pubchem (cid)**
* **Calculating fingerprints for each chemicel**
* **Functions**
  * `get_smiles_cid`: *parsing smiles from pubchem using cid*
  * `get_smiles_name`: *parsing smiles from pubchem using name*
  * `calculate_morgan_fingerprint`: *estimating fingerprints for smiles with give radius and bits*

In [59]:
def get_smiles_name(drug_name: str):
    try:
        compound = pcp.get_compounds(drug_name, 'name')[0]
        return compound.canonical_smiles
    except:
        return None
    

def get_smiles_id(pubchem_id: int):
    try:
        compound = pcp.get_compounds(pubchem_id, 'cid')[0]
        return compound.canonical_smiles
    except:
        return None


def calculate_morgan_fingerprint(smiles, radius, nBits):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * nBits
    else:
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits))


def df_na(df):
    # Select rows that contain at least one NA value
    rows_with_na = df[df.isnull().any(axis=1)]

    # Select columns that contain at least one NA value in those rows
    columns_with_na = rows_with_na.columns[rows_with_na.isnull().any()]

    # Show the selected rows and columns
    result = rows_with_na[columns_with_na]

    return result

In [45]:
# usage of pubchempy library
drug_names = ["Erlotinib", "Rapamycin", "Sunitinib"]
cids = ["176870", "5284616", "5329102"]

In [46]:
# get SMILES using drug name
for drug_name in drug_names:
    try:
        compound = pcp.get_compounds(drug_name, 'name')[0]
        smiles = compound.canonical_smiles
        print(f"The SMILES for {drug_name} is: {smiles}")
    except Exception as e:
        print(f"Could not retrieve data for {drug_name}")

The SMILES for Erlotinib is: COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C)OCCOC
The SMILES for Rapamycin is: CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)CC(OC(=O)C3CCCCN3C(=O)C(=O)C1(O2)O)C(C)CC4CCC(C(C4)OC)O)C)C)O)OC)C)C)C)OC
The SMILES for Sunitinib is: CCN(CC)CCNC(=O)C1=C(NC(=C1C)C=C2C3=C(C=CC(=C3)F)NC2=O)C


In [47]:
# get SMILES using pubchem id
for cid in cids:
    try:
        compound = pcp.get_compounds(cid, 'cid')[0]
        smiles = compound.canonical_smiles
        print(f"The SMILES for {cid} is: {smiles}")
    except Exception as e:
        print(f"Could not retrieve data for {cid}")

The SMILES for 176870 is: COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C)OCCOC
The SMILES for 5284616 is: CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)CC(OC(=O)C3CCCCN3C(=O)C(=O)C1(O2)O)C(C)CC4CCC(C(C4)OC)O)C)C)O)OC)C)C)C)OC
The SMILES for 5329102 is: CCN(CC)CCNC(=O)C1=C(NC(=C1C)C=C2C3=C(C=CC(=C3)F)NC2=O)C


* **parsing the smiles from pubchem**

In [41]:
df_drugs['SMILES'] = df_drugs['pubchem'].apply(get_smiles_id)

In [49]:
df_drugs

,pubchem,SMILES
0,176870,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...
1,5384616,C1=CC=C(C=C1)N=C(NC#N)[N]C#N
2,5329102,CCN(CC)CCNC(=O)C1=C(NC(=C1C)C=C2C3=C(C=CC(=C3)...
3,10461815,CC1=C(NC(=C1C(=O)N2CCCC2CN3CCCC3)C)C=C4C5=C(C=...
4,462382,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...
...,...,...
302,16224745,CN1C=C(C=N1)C2=C3N=C(C(=C(N3N=C2)N)Br)C4CCCNC4
303,118704762,C1COCCN1C2=CC=C(C=C2)C3=C(C=NC=C3)C4=CC(=C(C(=...
304,91663328,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...
305,117072552,C1CC(C1)NC2=NC=CC(=C2)C(=O)NCC(CN3CCC4=CC=CC=C...


In [61]:
df_na(df_drugs)

""


In [51]:
fingerprint_data = df_drugs['SMILES'].apply(calculate_morgan_fingerprint, radius=3, nBits=1024)
fingerprint_data

0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
                             ...                        
302    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
303    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
304    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
305    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
306    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: SMILES, Length: 307, dtype: object

* **fingerprint dataframe**

In [55]:
fingerprint_df = pd.DataFrame(fingerprint_data.tolist(), columns=[f'mfp{i+1}' for i in range(1024)], index=df_drugs.index.values)
fingerprint_df

,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,mfp7,mfp8,mfp9,mfp10,...,mfp1015,mfp1016,mfp1017,mfp1018,mfp1019,mfp1020,mfp1021,mfp1022,mfp1023,mfp1024
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
303,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
304,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
305,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


* **final chemical fingerprint**

In [62]:
df_drug_fingerprint = pd.concat([df_drugs, fingerprint_df] , axis=1)
df_drug_fingerprint

,pubchem,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,mfp7,mfp8,...,mfp1015,mfp1016,mfp1017,mfp1018,mfp1019,mfp1020,mfp1021,mfp1022,mfp1023,mfp1024
0,176870,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5384616,C1=CC=C(C=C1)N=C(NC#N)[N]C#N,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329102,CCN(CC)CCNC(=O)C1=C(NC(=C1C)C=C2C3=C(C=CC(=C3)...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10461815,CC1=C(NC(=C1C(=O)N2CCCC2CN3CCCC3)C)C=C4C5=C(C=...,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,462382,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,16224745,CN1C=C(C=N1)C2=C3N=C(C(=C(N3N=C2)N)Br)C4CCCNC4,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
303,118704762,C1COCCN1C2=CC=C(C=C2)C3=C(C=NC=C3)C4=CC(=C(C(=...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
304,91663328,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
305,117072552,C1CC(C1)NC2=NC=CC(=C2)C(=O)NCC(CN3CCC4=CC=CC=C...,0,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0


## Step 5: Merging the chemical features
- merging drug response data with newly estimated drug fingerprint data
- DRUG RESPONSE + CHEMICAL FEATURES


In [58]:
df_response

,COSMIC.ID,pubchem,response
0,909762,176870,0
1,910691,176870,0
2,910697,176870,1
3,1303901,176870,0
4,909248,176870,0
...,...,...,...
170197,749711,46907787,1
170198,906826,46907787,0
170199,906851,46907787,0
170200,925338,46907787,0


In [63]:
df_drug_fingerprint

,pubchem,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,mfp7,mfp8,...,mfp1015,mfp1016,mfp1017,mfp1018,mfp1019,mfp1020,mfp1021,mfp1022,mfp1023,mfp1024
0,176870,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5384616,C1=CC=C(C=C1)N=C(NC#N)[N]C#N,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5329102,CCN(CC)CCNC(=O)C1=C(NC(=C1C)C=C2C3=C(C=CC(=C3)...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10461815,CC1=C(NC(=C1C(=O)N2CCCC2CN3CCCC3)C)C=C4C5=C(C=...,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,462382,CC(C)CC(C=O)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=...,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,16224745,CN1C=C(C=N1)C2=C3N=C(C(=C(N3N=C2)N)Br)C4CCCNC4,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
303,118704762,C1COCCN1C2=CC=C(C=C2)C3=C(C=NC=C3)C4=CC(=C(C(=...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
304,91663328,CC1CN(CC(N1)C)C2=NC=C(C(=C2)C)C3=CC=C(C=C3)C4=...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
305,117072552,C1CC(C1)NC2=NC=CC(=C2)C(=O)NCC(CN3CCC4=CC=CC=C...,0,1,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [64]:
df_response_with_chem_features = df_response.merge(df_drug_fingerprint, on="pubchem", how="left")
df_response_with_chem_features

,COSMIC.ID,pubchem,response,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,...,mfp1015,mfp1016,mfp1017,mfp1018,mfp1019,mfp1020,mfp1021,mfp1022,mfp1023,mfp1024
0,909762,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,910691,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,910697,176870,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1303901,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,909248,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170197,749711,46907787,1,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170198,906826,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170199,906851,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170200,925338,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [65]:
df_na(df_response_with_chem_features)

""


## Step 6: Saving the final result data

* **Save and test**
  * save 
  * load and test 
---

In [66]:
save_file_chem = data_root.joinpath("drug_response_with_chemical.csv")
df_response_with_chem_features.to_csv(save_file_chem, index=False, header=True)

In [68]:
load_test_chem = pd.read_csv(save_file_chem)
load_test_chem

,COSMIC.ID,pubchem,response,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,...,mfp1015,mfp1016,mfp1017,mfp1018,mfp1019,mfp1020,mfp1021,mfp1022,mfp1023,mfp1024
0,909762,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,910691,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,910697,176870,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1303901,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,909248,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170197,749711,46907787,1,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170198,906826,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170199,906851,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170200,925338,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


---

## Step 7: Optional: Preparing cell line features

* **Creating genomic features using linear transformation using Cell Line IDs**
  * Applying linear transformation (PCA) on expression data
  * Create new celline genomic features dataframe   
  * Visualize and observe their relationship on latent space 
---

In [78]:
df_exp

,COSMIC.ID,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,H3C2,H3C3,AC098582.1,DUS4L.BCAP29,C8orf44.SGK3,ELOA3B,NPBWR1,ELOA3D,ELOA3,CDR1
0,684072,4.327687,0.070389,5.979339,2.906891,4.904484,0.263034,2.235727,0.422233,4.433627,...,1.416840,2.000000,0.042644,1.959770,0.432959,0.070389,0.042644,0.378512,0.070389,0.000000
1,687448,3.266037,0.000000,6.096979,2.521051,3.040892,0.000000,0.831877,6.576069,4.657068,...,0.333424,0.879706,0.839960,2.384050,0.000000,0.028569,0.163499,0.000000,0.028569,0.000000
2,687562,4.374344,0.000000,6.963821,2.292782,4.001802,0.495695,2.729009,6.396947,5.355792,...,2.056584,0.000000,0.333424,2.792855,0.584963,0.000000,0.238787,0.070389,0.000000,0.028569
3,687568,3.477677,0.000000,6.762615,2.107688,4.371559,0.028569,1.321928,5.440288,4.072963,...,1.910733,1.035624,0.367371,1.673556,0.097611,0.000000,0.042644,0.124328,0.000000,0.389567
4,687590,3.244887,0.000000,7.124535,1.933573,3.493135,0.014355,3.168321,6.090642,4.505891,...,1.695994,1.613532,0.594549,4.015248,0.286881,0.000000,0.028569,0.000000,0.000000,0.411426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,1330948,0.176323,0.000000,6.348197,2.280956,3.636915,1.831877,5.785551,5.497293,6.055065,...,1.937344,0.432959,0.454176,2.920293,0.189034,0.042644,0.014355,0.042644,0.042644,0.000000
658,1503364,4.585563,0.000000,7.524973,2.211012,2.786596,0.000000,0.310340,5.228819,4.791293,...,1.144046,0.782409,0.263034,1.220330,0.111031,0.000000,0.000000,0.000000,0.000000,0.000000
659,1659818,0.137504,0.000000,6.757423,3.176323,5.111449,0.097611,0.189034,3.688180,4.213347,...,0.650765,1.422233,0.604071,2.223423,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
660,1659819,0.000000,0.000000,7.059939,2.761285,4.144862,0.443607,0.000000,0.056584,4.491853,...,1.443607,1.906891,1.070389,1.448901,0.641546,0.000000,0.000000,0.000000,0.000000,0.000000


#### Applying PCA

In [79]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Step 1: drop cosmic_id and create data for PCA
cosmic_id = df_exp['COSMIC.ID']
data = df_exp.drop(columns=['COSMIC.ID'])

In [80]:
# Step 2: Standardize the data
data_standardized = StandardScaler().fit_transform(data)

# Step 3: Perform PCA
n_components = 10
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(data_standardized)

In [81]:
# Step 4: Create a new DataFrame with COSMIC.ID as a column and the principal components as additional columns
df_pc = pd.DataFrame(
    data=principal_components,
    index=df_exp.index.values,
    columns=[f"pc{i+1}" for i in range(n_components)],
)

In [82]:
df_pc

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,-78.075827,9.887937,2.218793,27.209164,-11.211991,2.043574,-23.638087,-16.682566,-0.904396,1.425845
1,-0.953831,-30.425885,34.816011,2.743034,29.783227,48.913967,24.521019,24.569328,-18.045163,21.509599
2,-25.191410,-27.264186,20.828492,-4.555884,31.931106,0.410029,9.636726,2.504808,2.754962,1.589406
3,-30.057025,-22.034219,35.916045,-5.134924,-14.856273,-9.519375,-8.307507,6.473962,-4.907480,-5.708629
4,-9.962854,-37.112092,54.046048,-5.959713,15.587697,3.236520,18.419816,7.436515,-4.759407,-15.122963
...,...,...,...,...,...,...,...,...,...,...
657,-24.800509,54.888704,13.434091,-15.031014,5.622916,6.990330,-16.765797,-25.019976,-8.172198,29.286098
658,97.407966,8.080495,-16.836750,31.098680,-5.171324,-22.682291,31.963529,-5.084064,4.449504,10.661608
659,-9.858470,57.282751,12.119594,-22.406916,-5.376340,15.850298,12.974977,37.981240,8.766231,-5.338135
660,-13.639014,89.117852,14.886149,-5.622675,-14.781503,-39.734083,24.906890,24.152851,36.580533,-0.047079


In [84]:
df_exp_features = pd.concat([cosmic_id, df_pc], axis=1)
df_exp_features

,COSMIC.ID,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,684072,-78.075827,9.887937,2.218793,27.209164,-11.211991,2.043574,-23.638087,-16.682566,-0.904396,1.425845
1,687448,-0.953831,-30.425885,34.816011,2.743034,29.783227,48.913967,24.521019,24.569328,-18.045163,21.509599
2,687562,-25.191410,-27.264186,20.828492,-4.555884,31.931106,0.410029,9.636726,2.504808,2.754962,1.589406
3,687568,-30.057025,-22.034219,35.916045,-5.134924,-14.856273,-9.519375,-8.307507,6.473962,-4.907480,-5.708629
4,687590,-9.962854,-37.112092,54.046048,-5.959713,15.587697,3.236520,18.419816,7.436515,-4.759407,-15.122963
...,...,...,...,...,...,...,...,...,...,...,...
657,1330948,-24.800509,54.888704,13.434091,-15.031014,5.622916,6.990330,-16.765797,-25.019976,-8.172198,29.286098
658,1503364,97.407966,8.080495,-16.836750,31.098680,-5.171324,-22.682291,31.963529,-5.084064,4.449504,10.661608
659,1659818,-9.858470,57.282751,12.119594,-22.406916,-5.376340,15.850298,12.974977,37.981240,8.766231,-5.338135
660,1659819,-13.639014,89.117852,14.886149,-5.622675,-14.781503,-39.734083,24.906890,24.152851,36.580533,-0.047079


### Lets explore this dimension reduction visually

In [86]:
cell_info_file = data_root.joinpath("cell_info_filtered_662cell.csv")
df_cell_info = pd.read_csv(cell_info_file)

In [87]:
df_cell_info

,Cell.Line.Name,COSMIC.ID,GDSC.Desc1,GDSC.Desc2,TCGA.Desc
0,YKG-1,687592,nervous_system,glioma,GBM
1,Calu-3,687777,lung,lung_NSCLC_adenocarcinoma,LUAD
2,42-MG-BA,687561,nervous_system,glioma,GBM
3,8-MG-BA,687562,nervous_system,glioma,GBM
4,GB-1,687568,nervous_system,glioma,GBM
...,...,...,...,...,...
657,MM1S,1659818,blood,myeloma,MM
658,OCI-LY7,1659819,blood,B_cell_lymphoma,DLBC
659,SNU-81,1660036,digestive_system,large_intestine,COREAD
660,SNU-1040,1659823,digestive_system,large_intestine,COREAD


In [88]:
df_exp_features

,COSMIC.ID,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,684072,-78.075827,9.887937,2.218793,27.209164,-11.211991,2.043574,-23.638087,-16.682566,-0.904396,1.425845
1,687448,-0.953831,-30.425885,34.816011,2.743034,29.783227,48.913967,24.521019,24.569328,-18.045163,21.509599
2,687562,-25.191410,-27.264186,20.828492,-4.555884,31.931106,0.410029,9.636726,2.504808,2.754962,1.589406
3,687568,-30.057025,-22.034219,35.916045,-5.134924,-14.856273,-9.519375,-8.307507,6.473962,-4.907480,-5.708629
4,687590,-9.962854,-37.112092,54.046048,-5.959713,15.587697,3.236520,18.419816,7.436515,-4.759407,-15.122963
...,...,...,...,...,...,...,...,...,...,...,...
657,1330948,-24.800509,54.888704,13.434091,-15.031014,5.622916,6.990330,-16.765797,-25.019976,-8.172198,29.286098
658,1503364,97.407966,8.080495,-16.836750,31.098680,-5.171324,-22.682291,31.963529,-5.084064,4.449504,10.661608
659,1659818,-9.858470,57.282751,12.119594,-22.406916,-5.376340,15.850298,12.974977,37.981240,8.766231,-5.338135
660,1659819,-13.639014,89.117852,14.886149,-5.622675,-14.781503,-39.734083,24.906890,24.152851,36.580533,-0.047079


In [91]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource
import umap
import pandas as pd
from bokeh.palettes import viridis
from bokeh.transform import linear_cmap
from bokeh.layouts import column

def plot_umap(df_exp_features, df_cell_info, desc_columns=['TCGA.Desc', 'GDSC.Desc1', 'GDSC.Desc2']):
    # Step 1: Merge dataframes
    df_merged = pd.merge(df_exp_features, df_cell_info, on='COSMIC.ID')
    
    # Step 2: Apply UMAP
    features = df_merged.loc[:, 'pc1':'pc10']
    umap_transformer = umap.UMAP()
    umap_result = umap_transformer.fit_transform(features)
    
    df_merged['umap_x'] = umap_result[:, 0]
    df_merged['umap_y'] = umap_result[:, 1]
    
    # Step 3: Create Bokeh plot
    source = ColumnDataSource(df_merged)
    
    mapper = linear_cmap(field_name='umap_x', palette=viridis(256), low=min(df_merged['umap_x']), high=max(df_merged['umap_x']))

    hover = HoverTool(
        tooltips=[
            ('Cell Line Name', '@{Cell.Line.Name}'), 
            ('COSMIC ID', '@{COSMIC.ID}'),
            ('TCGA.Desc', '@{TCGA.Desc}'),
            ('GDSC.Desc1', '@{GDSC.Desc1}'),
            ('GDSC.Desc2', '@{GDSC.Desc2}'),
        ]
    )

    p = figure(width=600, height=600, tools=[hover], title="UMAP visualization of Cell Lines")
    p.circle('umap_x', 'umap_y', source=source, size=5, color=mapper, alpha=0.6)

    output_notebook()
    show(column(p))

In [93]:
plot_umap(df_exp_features, df_cell_info)

Loading BokehJS ...

## Step 8: Mapping the features into final training data

* **Merging -> CHEMICAL_FEATURES + CELL LINE FEATURES**

In [95]:
save_file_chem

PosixPath('input_data/drug_response_with_chemical.csv')

In [96]:
# loading chemical features
df_chemical_features = pd.read_csv(save_file_chem)
df_chemical_features

,COSMIC.ID,pubchem,response,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,...,mfp1015,mfp1016,mfp1017,mfp1018,mfp1019,mfp1020,mfp1021,mfp1022,mfp1023,mfp1024
0,909762,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,910691,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,910697,176870,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1303901,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,909248,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170197,749711,46907787,1,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170198,906826,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170199,906851,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
170200,925338,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [97]:
df_exp_features

,COSMIC.ID,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,684072,-78.075827,9.887937,2.218793,27.209164,-11.211991,2.043574,-23.638087,-16.682566,-0.904396,1.425845
1,687448,-0.953831,-30.425885,34.816011,2.743034,29.783227,48.913967,24.521019,24.569328,-18.045163,21.509599
2,687562,-25.191410,-27.264186,20.828492,-4.555884,31.931106,0.410029,9.636726,2.504808,2.754962,1.589406
3,687568,-30.057025,-22.034219,35.916045,-5.134924,-14.856273,-9.519375,-8.307507,6.473962,-4.907480,-5.708629
4,687590,-9.962854,-37.112092,54.046048,-5.959713,15.587697,3.236520,18.419816,7.436515,-4.759407,-15.122963
...,...,...,...,...,...,...,...,...,...,...,...
657,1330948,-24.800509,54.888704,13.434091,-15.031014,5.622916,6.990330,-16.765797,-25.019976,-8.172198,29.286098
658,1503364,97.407966,8.080495,-16.836750,31.098680,-5.171324,-22.682291,31.963529,-5.084064,4.449504,10.661608
659,1659818,-9.858470,57.282751,12.119594,-22.406916,-5.376340,15.850298,12.974977,37.981240,8.766231,-5.338135
660,1659819,-13.639014,89.117852,14.886149,-5.622675,-14.781503,-39.734083,24.906890,24.152851,36.580533,-0.047079


In [98]:
# df_response.merge(df_drug_fingerprint, on="pubchem", how="left")
df_final_data = df_chemical_features.merge(df_exp_features, on="COSMIC.ID", how='left')

In [99]:
df_final_data

,COSMIC.ID,pubchem,response,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,...,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,909762,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,-51.630190,79.186577,6.050413,-15.675212,15.584058,-37.377713,-6.733680,-18.253078,-5.970032,13.568325
1,910691,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,-68.582726,6.593351,-30.644074,66.491235,13.252894,17.803318,-19.692284,-0.785197,-6.073031,-15.700322
2,910697,176870,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,84.828565,-12.435305,2.586667,12.131196,-12.059170,-34.175637,2.804909,17.764811,-7.526032,21.535021
3,1303901,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,-109.033026,9.857870,-44.537184,58.005796,-7.171131,-37.021263,20.098246,-1.699299,-17.892506,9.978079
4,909248,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,12.814317,-16.181661,1.426233,12.069189,33.424655,7.938392,-7.770647,0.607013,-3.593249,-11.629081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170197,749711,46907787,1,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,-24.615410,-12.192057,-27.749511,-1.226036,24.247485,-4.448370,-26.584927,6.820419,0.320980,15.552473
170198,906826,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,8.352528,-22.379867,23.277306,5.678401,-7.232523,-9.673819,-2.692808,-9.420698,13.973877,-3.919699
170199,906851,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,8.178329,1.897205,-45.340319,2.878842,4.333707,14.646394,-31.926380,8.946441,19.298089,20.263891
170200,925338,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,-26.504405,-22.077314,22.655068,21.192186,17.991488,-3.774143,-18.581248,-6.538313,20.036636,5.641548


## Step 9: Saving the final result data

* **Save and test**
  * save 
  * load and test 
---

In [101]:
save_file_final = data_root.joinpath("final_data.csv")
df_final_data.to_csv(save_file_final, index=False, header=True)

In [102]:
load_test_final = pd.read_csv(save_file_final)
load_test_final

,COSMIC.ID,pubchem,response,SMILES,mfp1,mfp2,mfp3,mfp4,mfp5,mfp6,...,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
0,909762,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,-51.630190,79.186577,6.050413,-15.675212,15.584058,-37.377713,-6.733680,-18.253078,-5.970032,13.568325
1,910691,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,-68.582726,6.593351,-30.644074,66.491235,13.252894,17.803318,-19.692284,-0.785197,-6.073031,-15.700322
2,910697,176870,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,84.828565,-12.435305,2.586667,12.131196,-12.059170,-34.175637,2.804909,17.764811,-7.526032,21.535021
3,1303901,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,-109.033026,9.857870,-44.537184,58.005796,-7.171131,-37.021263,20.098246,-1.699299,-17.892506,9.978079
4,909248,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,0,0,0,0,0,0,...,12.814317,-16.181661,1.426233,12.069189,33.424655,7.938392,-7.770647,0.607013,-3.593249,-11.629081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170197,749711,46907787,1,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,-24.615410,-12.192057,-27.749511,-1.226036,24.247485,-4.448370,-26.584927,6.820419,0.320980,15.552473
170198,906826,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,8.352528,-22.379867,23.277306,5.678401,-7.232523,-9.673819,-2.692808,-9.420698,13.973877,-3.919699
170199,906851,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,8.178329,1.897205,-45.340319,2.878842,4.333707,14.646394,-31.926380,8.946441,19.298089,20.263891
170200,925338,46907787,0,CC1=C(SC2=C1C(=NC(C3=NN=C(N32)C)CC(=O)OC(C)(C)...,0,0,0,0,0,0,...,-26.504405,-22.077314,22.655068,21.192186,17.991488,-3.774143,-18.581248,-6.538313,20.036636,5.641548


## Step 10: Homework

* **Get smiles from drugbank data "input_data/structure.sdf"**
* **Dimension reduction onf mutation data"**

### END HANDS-ON 
---